In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn import svm 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [2]:
df=pd.read_csv("train_df.csv")
df.head()

/home/attributy-njomza/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,LoanID,ApplicantID,ApplicantGender,ApplicantMarried,ApplicantDependents,ApplicantEducation,ApplicantSelfEmployed,ApplicantIncome,ApplicantCreditHistory,ApplicantZIP,...,ApplicantEmplLength,ApplicantHomeOwn,LoanAmount,LoanTerm,LoanIntRate,LoanDesc,LoanPurpose,LoanApproved,Unnamed: 19,Unnamed: 20
0,56026,19874,Moški,NE,0 oseb,Diplomiral,NE,1949.666667,1.0,33xx,...,5 let,STANOVANJSKI KREDIT,NaN,90.0,1.42%,NaN,prenova,Y,NaN,NaN
1,53884,34258,Moški,DA,1 oseba,Diplomiral,NE,1527.666667,1.0,64xx,...,10+ let,NAJEM,44800.0,90.0,3.90%,NaN,drugo,N,NaN,NaN
2,48350,66552,Moški,DA,0 oseb,Brez diplome,NE,861.000000,1.0,30xx,...,3 leta,NAJEM,42000.0,90.0,2.80%,NaN,NaN,Y,NaN,NaN
3,30455,54844,Moški,DA,2 osebi,Diplomiral,NE,1066.666667,1.0,84xx,...,1 leto,NAJEM,24500.0,90.0,3.26%,NaN,NaN,Y,NaN,NaN
4,935,18820,Moški,DA,2 osebi,Diplomiral,NaN,833.333333,1.0,60xx,...,10+ let,STANOVANJSKI KREDIT,38150.0,90.0,3.26%,NaN,NaN,Y,NaN,NaN


In [3]:
print("Data columns info: \n")
df.info(verbose = True)

Data columns info: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38767 entries, 0 to 38766
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   LoanID                  38767 non-null  int64  
 1   ApplicantID             38767 non-null  int64  
 2   ApplicantGender         37902 non-null  object 
 3   ApplicantMarried        38573 non-null  object 
 4   ApplicantDependents     37775 non-null  object 
 5   ApplicantEducation      38767 non-null  object 
 6   ApplicantSelfEmployed   36702 non-null  object 
 7   ApplicantIncome         38767 non-null  float64
 8   ApplicantCreditHistory  35687 non-null  float64
 9   ApplicantZIP            38767 non-null  object 
 10  ApplicantState          38767 non-null  object 
 11  ApplicantEmplLength     36924 non-null  object 
 12  ApplicantHomeOwn        38767 non-null  object 
 13  LoanAmount              37370 non-null  float64
 14  LoanTerm         

In [4]:
df.isnull().sum()

LoanID                        0
ApplicantID                   0
ApplicantGender             865
ApplicantMarried            194
ApplicantDependents         992
ApplicantEducation            0
ApplicantSelfEmployed      2065
ApplicantIncome               0
ApplicantCreditHistory     3080
ApplicantZIP                  0
ApplicantState                0
ApplicantEmplLength        1843
ApplicantHomeOwn              0
LoanAmount                 1397
LoanTerm                    904
LoanIntRate                   0
LoanDesc                  37246
LoanPurpose               23912
LoanApproved                  8
Unnamed: 19               38752
Unnamed: 20               38763
dtype: int64

In [5]:
df.duplicated().any()

False

In [6]:
df['ApplicantGender']=df['ApplicantGender'].fillna("Unkown")
df['ApplicantGender'].unique()
df['ApplicantMarried'].fillna(df['ApplicantMarried'].mode()[0], inplace=True)
df['ApplicantDependents'].fillna(df['ApplicantDependents'].mode()[0], inplace=True)
df['ApplicantSelfEmployed'].fillna(df['ApplicantSelfEmployed'].mode()[0], inplace=True)
df['ApplicantCreditHistory'].fillna(df['ApplicantCreditHistory'].mode()[0],inplace=True)
df['ApplicantEmplLength'].fillna(df['ApplicantEmplLength'].mode()[0], inplace=True)
df['LoanAmount'].fillna(df['LoanAmount'].mode()[0],inplace=True)
df['LoanTerm'].fillna(df['LoanTerm'].mode()[0], inplace=True)

df.drop(['LoanPurpose', 'LoanDesc','ApplicantState','LoanID','ApplicantID'], axis=1, inplace=True)

In [7]:
df.isnull().sum()

ApplicantGender               0
ApplicantMarried              0
ApplicantDependents           0
ApplicantEducation            0
ApplicantSelfEmployed         0
ApplicantIncome               0
ApplicantCreditHistory        0
ApplicantZIP                  0
ApplicantEmplLength           0
ApplicantHomeOwn              0
LoanAmount                    0
LoanTerm                      0
LoanIntRate                   0
LoanApproved                  8
Unnamed: 19               38752
Unnamed: 20               38763
dtype: int64

In [8]:
df.dtypes

ApplicantGender            object
ApplicantMarried           object
ApplicantDependents        object
ApplicantEducation         object
ApplicantSelfEmployed      object
ApplicantIncome           float64
ApplicantCreditHistory    float64
ApplicantZIP               object
ApplicantEmplLength        object
ApplicantHomeOwn           object
LoanAmount                float64
LoanTerm                  float64
LoanIntRate                object
LoanApproved               object
Unnamed: 19                object
Unnamed: 20                object
dtype: object

In [9]:
df.replace({ 'ApplicantDependents':{'0 oseb':0,'1 oseba':1,'2 osebi':2, '3+ osebe':3},
             'ApplicantEmplLength':{'1 leto':1,'2 leti':2,'< 1 leto':0.5,'3 leta':3, '4 leta':4,'5 let':5,'6 let':6,'7 let':7,'8 let':8,'9 let':9,'10+ let':10}},inplace=True)

df['LoanIntRate'] = df['LoanIntRate'].str.replace('%','')
def converting(x):
    return float(x)/100
df['LoanIntRate']=df['LoanIntRate'].map(converting)

df.dtypes

ApplicantGender            object
ApplicantMarried           object
ApplicantDependents         int64
ApplicantEducation         object
ApplicantSelfEmployed      object
ApplicantIncome           float64
ApplicantCreditHistory    float64
ApplicantZIP               object
ApplicantEmplLength       float64
ApplicantHomeOwn           object
LoanAmount                float64
LoanTerm                  float64
LoanIntRate               float64
LoanApproved               object
Unnamed: 19                object
Unnamed: 20                object
dtype: object

In [10]:
categorical_cols = ["ApplicantGender", "ApplicantMarried", "ApplicantEducation", "ApplicantSelfEmployed",
                    "ApplicantHomeOwn", "ApplicantZIP","LoanApproved"]

continous_cols = [ "ApplicantDependents",
                  "ApplicantIncome", "ApplicantCreditHistory", "ApplicantEmplLength","LoanAmount"
                 ,"LoanTerm","LoanIntRate"]

for col in categorical_cols:
    df[col] = df[col].astype("category")
    
for col in continous_cols:
    df[col] = df[col].astype("int16")

In [11]:
df.dtypes

ApplicantGender           category
ApplicantMarried          category
ApplicantDependents          int16
ApplicantEducation        category
ApplicantSelfEmployed     category
ApplicantIncome              int16
ApplicantCreditHistory       int16
ApplicantZIP              category
ApplicantEmplLength          int16
ApplicantHomeOwn          category
LoanAmount                   int16
LoanTerm                     int16
LoanIntRate                  int16
LoanApproved              category
Unnamed: 19                 object
Unnamed: 20                 object
dtype: object

In [12]:
for col in categorical_cols:
    print(f"Unique values of {col} column is : {list(df[col].unique())}")

Unique values of ApplicantGender column is : ['Moški', 'Unkown', 'Ženska']
Unique values of ApplicantMarried column is : ['NE', 'DA']
Unique values of ApplicantEducation column is : ['Diplomiral', 'Brez diplome']
Unique values of ApplicantSelfEmployed column is : ['NE', 'DA']
Unique values of ApplicantHomeOwn column is : ['STANOVANJSKI KREDIT', 'NAJEM', 'LASTNO']
Unique values of ApplicantZIP column is : ['33xx', '64xx', '30xx', '84xx', '60xx', '62xx', '11xx', '20xx', '63xx', '53xx', '74xx', '44xx', '43xx', '23xx', '41xx', '80xx', '40xx', '32xx', '21xx', '03xx', '02xx', '72xx', '24xx', '14xx', '10xx', '13xx', '34xx', '12xx', '22xx', '70xx', '54xx', '01xx', '71xx', '31xx', '81xx', '73xx', '61xx', '42xx', '82xx', '52xx', '83xx', '00xx', '51xx', '04xx']
Unique values of LoanApproved column is : ['Y', 'N', nan, ' sklad. <br>', 'kartica', 'air za hi\x9ao <br>', 'prenova', ' prebarvanje izven hi\x9ae in prebarvanje celotno notranjost hi\x9ae <br>Izjemna terasa in postavite terasa pokrov <br>

In [13]:
def one_hot_encode(data, cols, postfix= "_new") -> pd.DataFrame:
   
    global new_cols
    new_cols = [col + "_new" for col in cols]
    
    for new_col, col in zip(new_cols, cols):
        data[new_col] = data[col].cat.codes
        
    data = data.drop(cols, axis= 1)
    return data

df = one_hot_encode(df, categorical_cols)

In [14]:
for col in new_cols:
    print(f"Unique values for {col} column after "+ \
          f"one hot encoding is: {list(df[col].unique())}")

Unique values for ApplicantGender_new column after one hot encoding is: [0, 1, 2]
Unique values for ApplicantMarried_new column after one hot encoding is: [1, 0]
Unique values for ApplicantEducation_new column after one hot encoding is: [1, 0]
Unique values for ApplicantSelfEmployed_new column after one hot encoding is: [1, 0]
Unique values for ApplicantHomeOwn_new column after one hot encoding is: [2, 1, 0]
Unique values for ApplicantZIP_new column after one hot encoding is: [18, 33, 15, 43, 29, 31, 6, 10, 32, 27, 38, 24, 23, 13, 21, 39, 20, 17, 11, 3, 2, 36, 14, 9, 5, 8, 19, 7, 12, 34, 28, 1, 35, 16, 40, 37, 30, 22, 41, 26, 42, 0, 25, 4]
Unique values for LoanApproved_new column after one hot encoding is: [4, 3, -1, 2, 6, 5, 7, 1, 0]


In [15]:
def z_score(df: pd.DataFrame, col_name: str) -> pd.DataFrame:
 
    
    std: np.float16 = np.float16(df[col_name].std())
    mean: np.float16 = np.float16(df[col_name].mean())
    
    upper_limit = mean + (3 * std)
    lower_limit = mean - (3 * std)
    
    outliers_free = [value for value in df[col_name] if 
                  not((value > upper_limit) | (value < lower_limit))]
    

    filtered_data = df.loc[df[col_name].isin(outliers_free)]
    
    
    return filtered_data

In [16]:
X=df.drop(columns=['LoanApproved_new'],axis=1)
print(X)

y=df['LoanApproved_new']
print(y)

       ApplicantDependents  ApplicantIncome  ApplicantCreditHistory  \
0                        0             1949                       1   
1                        1             1527                       1   
2                        0              861                       1   
3                        2             1066                       1   
4                        2              833                       1   
...                    ...              ...                     ...   
38762                    1              873                       1   
38763                    0             3107                       1   
38764                    0              800                       0   
38765                    3             1365                       1   
38766                    2               70                       1   

       ApplicantEmplLength  LoanAmount  LoanTerm  LoanIntRate Unnamed: 19  \
0                        5      -23536        90            0         

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [18]:
model = LogisticRegression()
model.fit(X_train, y_train)
print(type(X_test))
ypred = model.predict(X_test)

print(ypred)

ValueError: could not convert string to float: 'prenova'

In [19]:
accuracy = f1_score(y_test, ypred)
accuracy

NameError: name 'ypred' is not defined